In [2]:
import os
import csv
import re

from datetime import datetime

- test-folder
    - category_1
        - manual_1
            - manual_1.pdf
        - product_1
            - accessory_1
                - manual_4
                    - manual_4.pdf
            - manual_2
                - manual_2.pdf
            - manual_3
                - manual_3.pdf
        - product_2
            - manual_5
                - manual_5.pdf
            - manual_6
                - manual_6.pdf
            - manual_7
                - manual_7.pdf
    - category_2
        - manual_8
            - manual_8.pdf
        - product_3
            - accessory_2
                - manual_11
                - manual_12
            - manual_9
                - manual_9.pdf
            - manual_10
                - manual_10.pdf

In [13]:
for root, dirs, files in os.walk(os.path.join('target-folder', 'English_EN')):
    print(root, "|||", dirs, "|||", files)

target-folder\English_EN ||| ['MaxiCharger AC', 'MaxiCharger DC'] ||| ['EN_verification.csv']
target-folder\English_EN\MaxiCharger AC ||| ['Energy Management System', 'Compact', 'Ultra', 'Wallbox', 'Maintenance Manual'] ||| []
target-folder\English_EN\MaxiCharger AC\Energy Management System ||| ['Installation and Operation Manual'] ||| []
target-folder\English_EN\MaxiCharger AC\Energy Management System\Installation and Operation Manual ||| ['arch'] ||| ['Energy Management System - Operation and Installation Manual EU_20230525_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Energy Management System\Installation and Operation Manual\arch ||| [] ||| []
target-folder\English_EN\MaxiCharger AC\Compact ||| ['User Manual', 'Quick Guide'] ||| []
target-folder\English_EN\MaxiCharger AC\Compact\User Manual ||| ['arch'] ||| ['MaxiCharger AC Compact - User Manual_20230302_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Compact\User Manual\arch ||| [] ||| []
target-folder\English_EN\MaxiCharger A

In [14]:
# 目标文件夹根目录
target_folder = "target-folder"  

# 获取所有语言分类子文件夹的路径
#lang_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]
#lang_folders.remove('English_EN')

lang_folders = ['English_EN']
lang_folder = lang_folders[0]
lang_code = lang_folder.split("_")[-1]    # EN

csv_file = f"{lang_code}_log.csv"  # CSV文件名
csv_path = os.path.join(target_folder, lang_folder, csv_file)

"""
"""

# 打开CSV文件并写入表头
with open(csv_path, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["language", "category", "object", "document", "update", "status"])

    print(csv_path)

target-folder\English_EN\EN_log.csv


In [ ]:

    # 遍历每个文档名称文件夹
    for root, dirs, files in os.walk(os.path.join(target_folder, lang_folder)):    

        if "arch" in dirs:  # 只处理包含arch子文件夹的文档名称文件夹
            doc_folder = os.path.join(root, folder)
            doc_name = folder.replace("arch", "").strip()  # 提取文档名称
            product_name = os.path.basename(os.path.dirname(doc_folder))  # 提取产品名称
            product_category = os.path.basename(os.path.dirname(os.path.dirname(doc_folder)))  # 提取产品分类

            pdf_files = [file for file in files if file.lower().endswith(".pdf")]
            if pdf_files:
                latest_date = max([datetime.strptime(file.split("_")[2], "%Y%m%d") for file in pdf_files])
                status = "up-to-date" if lang_folder == "English_EN" else "out-of-date"
                latest_date_str = datetime.strftime(latest_date, "%Y%m%d")
            else:
                latest_date_str = "null"
                status = "null"

            writer.writerow([lang_code, product_category, product_name, doc_name, latest_date_str, status])

In [ ]:
# 遍历每个语言分类子文件夹
for lang_folder in language_folders:
    lang_code = lang_folder.split("_")[-1]  # 提取语言代码 EN, FR, DE

    csv_file = f"{lang_code}_verification.csv"  # CSV文件名
    csv_path = os.path.join(target_folder, lang_folder, csv_file)

    # 打开CSV文件并写入表头
    with open(csv_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["language", "category", "object", "document", "update", "status"])

        # 遍历每个文档名称文件夹
        for root, dirs, files in os.walk(os.path.join(target_folder, lang_folder)):    
            # 'target-folder\\English_EN'
            # ('target-folder\\English_EN', ['MaxiCharger AC', 'MaxiCharger DC'], ['EN_verification.csv'])
            for folder in dirs:
                if "arch" in folder:  # 只处理包含arch子文件夹的文档名称文件夹
                    doc_folder = os.path.join(root, folder)
                    doc_name = folder.replace("arch", "").strip()  # 提取文档名称
                    product_name = os.path.basename(os.path.dirname(doc_folder))  # 提取产品名称
                    product_category = os.path.basename(os.path.dirname(os.path.dirname(doc_folder)))  # 提取产品分类

                    pdf_files = [file for file in files if file.lower().endswith(".pdf")]
                    if pdf_files:
                        latest_date = max([datetime.strptime(file.split("_")[2], "%Y%m%d") for file in pdf_files])
                        status = "up-to-date" if lang_folder == "English_EN" else "out-of-date"
                        latest_date_str = datetime.strftime(latest_date, "%Y%m%d")
                    else:
                        latest_date_str = "null"
                        status = "null"

                    writer.writerow([lang_code, product_category, product_name, doc_name, latest_date_str, status])

In [ ]:

def folder_miner (dir, file_list):
    
    new_dir = dir

    if os.path.isfile(dir):
        file_list.append(dir)
    elif os.path.isdir(dir):
        for s in os.listdir(dir):
            new_dir = os.path.join(dir, s)
            folder_miner (new_dir, file_list)
    return file_list

if __name__=='__main__':
    l = folder_miner (root, [])

    print(len(l))

    for e in l:
        print(e)

In [19]:
os.listdir("material-folder")

['manuals', 'certification']

In [22]:
import datetime

# 当天日期
current_date = datetime.date.today().strftime("%Y%m%d")
# 语言变量
language = "EN"
# 文件名模板
filename_template = "file-name_{0}_{1}"
# 根据模板生成文件名
filename = filename_template.format(current_date, language)
print(filename)


file-name_20230702_EN


In [ ]:
import os

# 原始目录和目标目录
source_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"
target_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"

# 遍历源目录下的所有文件
for filename in os.listdir(source_dir):
    if filename.endswith(".pdf"):
        # 构建源文件路径和目标文件路径
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        
        # 将目标文件路径中的文件名替换为指定的 filename
        target_file = os.path.join(target_dir, filename.replace(".pdf", "_" + filename))
        
        # 重命名文件
        os.rename(source_file, target_file)